In [1]:
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)

In [2]:

filepath = ('C:/Users/andre/OneDrive/Desktop/GitHub/Health-AI/'
            'icr-identify-age-related-conditions/')

train   = pd.read_csv(filepath+'train.csv')
test    = pd.read_csv(filepath+'test.csv')
greeks  = pd.read_csv(filepath+'greeks.csv')
example = pd.read_csv(filepath+'sample_submission.csv')

In [3]:
#79############################################################################
#72#####################################################################
def prep_df(df):
    # Move categorical col to beginning of df
    cols = list(df.columns)
    cols.remove('EJ')
    # cols.insert(1,'EJ') # Temporarily I will remove this
    df=df[cols]
    
    df=df.set_index('Id').copy(deep=True)

    # Temporarily just fillna
    df.fillna(0,inplace=True)

    # Remove whitespace from column names
    renamer={}
    for col in df.columns:
        renamer[col]=col.strip()
    df.rename(columns=renamer,inplace=True)
    return df
train=prep_df(train)
test=prep_df(test)

In [4]:
# CLASSIFICATION RANDOM FOREST

import sklearn
from sklearn.tree import (
    DecisionTreeClassifier,
    DecisionTreeRegressor
    )
from sklearn.ensemble import (
    RandomForestClassifier,
    RandomForestRegressor,
    GradientBoostingClassifier
    )
from sklearn.model_selection import (
    train_test_split,
    cross_val_score,
    GridSearchCV
    )
import matplotlib.pylab as plt
from dmba import (
    plotDecisionTree,
    classificationSummary,
    regressionSummary
    )

# Separate into input variables and target variable.
x = train[train.columns[:-1]]
y = train[train.columns[-1]]


# Partition Data
train_x, valid_x, train_y, valid_y = train_test_split(x, y, test_size=.4, random_state=1)


# Create the tree.
rf = RandomForestClassifier(n_estimators=500, random_state=1)
# For RandomForestClassifier many parameters are of interest:
#     n_estimators:          number of trees in the forest.
#     random_state:          allows reproduceable results.
#     max_depth:             max number of levels below the root node.
#     min_impurity_decrease: min impurity decrease to split a node.
#     min_samples_split:     min samples to split a node.
rf.fit(train_x, train_y)


# # Variable (Feature) Importance Plot
# # Consider the Paramters: rotate, pdfFile
# importances = rf.feature_importances_
# std = np.std([tree.feature_importances_ for tree in rf.estimators_], axis=0)
# vip_df = pd.DataFrame({'feature': train_x.columns, 'importance': importances, 'std': std})
# vip_df.sort_values('importance', inplace=True, ascending=False)
# print(vip_df)
# vip_df.sort_values('importance', inplace=True, ascending=True)
# ax = vip_df.plot(kind='barh', xerr='std', x='feature', legend=False)
# ax.set_ylabel('')
# plt.show()


# Evaluate Performance
classificationSummary(train_y, rf.predict(train_x))
classificationSummary(valid_y, rf.predict(valid_x))

Confusion Matrix (Accuracy 1.0000)

       Prediction
Actual   0   1
     0 303   0
     1   0  67
Confusion Matrix (Accuracy 0.9231)

       Prediction
Actual   0   1
     0 202   4
     1  15  26


In [5]:
# Score test
"""
Predict_proba returns a 2 dimensional array, but I will just keep the
probabilities of class_1
"""
results = rf.predict_proba(test)[:,1]
submission = test.copy(deep=True)
submission['class_1'] = results
submission['class_0'] = 1 - results
submission=submission[['class_0','class_1']]